In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/non-fulfillment/sample_submission.csv
/kaggle/input/non-fulfillment/train.csv
/kaggle/input/non-fulfillment/test.csv


In [2]:
# Kaggle Notebook에서 AutoGluon 설치 (첫 실행 시 한 번만)
!pip install -q --upgrade autogluon
!pip install --upgrade scikit-learn==1.4.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.2/266.2 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.0/430.0 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.2/352.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.7/196.7 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.0/71.0 kB 5.7 MB/s eta 0:00

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier
import warnings
from autogluon.tabular import TabularPredictor
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE

In [4]:
# # 데이터 로드 및 전처리
# train_data = pd.read_csv('/kaggle/input/non-fulfillment/train.csv')
# test_data = pd.read_csv('/kaggle/input/non-fulfillment/test.csv')

# # UID 컬럼 제거
# train_data.drop(columns=['UID'], inplace=True)
# test_data.drop(columns=['UID'], inplace=True)

In [5]:
# 1. 데이터 로드 및 UID 제거
train_data = pd.read_csv('/kaggle/input/non-fulfillment/train.csv')
test_data = pd.read_csv('/kaggle/input/non-fulfillment/test.csv')
train_data.drop(columns=['UID'], inplace=True)
test_data.drop(columns=['UID'], inplace=True)

# 2. 범주형 변수 처리 (라벨 인코딩 & 원-핫 인코딩)
# 대상 변수: "주거 형태", "현재 직장 근속 연수", "대출 목적", "대출 상환 기간"
label_enc = LabelEncoder()
train_data["현재 직장 근속 연수"] = label_enc.fit_transform(train_data["현재 직장 근속 연수"])
test_data["현재 직장 근속 연수"] = label_enc.transform(test_data["현재 직장 근속 연수"])

train_data = pd.get_dummies(train_data, columns=["주거 형태", "대출 목적", "대출 상환 기간"], drop_first=True)
test_data = pd.get_dummies(test_data, columns=["주거 형태", "대출 목적", "대출 상환 기간"], drop_first=True)

# 3. 수치형 변수 변환 (로그 변환 및 "연체 없음" 컬럼 추가)
log_columns = ["현재 미상환 신용액", "월 상환 부채액", "현재 대출 잔액"]
for col in log_columns:
    train_data[col] = np.log1p(train_data[col])
    test_data[col] = np.log1p(test_data[col])

# "마지막 연체 이후 경과 개월 수"가 0이면 "연체 없음" 컬럼 생성
train_data["연체 없음"] = (train_data["마지막 연체 이후 경과 개월 수"] == 0).astype(int)
test_data["연체 없음"] = (test_data["마지막 연체 이후 경과 개월 수"] == 0).astype(int)

# 4. 결측값 처리 (SimpleImputer: 중간값으로 대체)
imputer = SimpleImputer(strategy="median")
# train_data의 경우, 타겟("채무 불이행 여부")과 특성을 분리하여 처리한 후 다시 합칩니다.
target = "채무 불이행 여부"
train_features = train_data.drop(columns=[target])
train_target = train_data[target]

train_features_imputed = pd.DataFrame(imputer.fit_transform(train_features), 
                                      columns=train_features.columns)
test_data = pd.DataFrame(imputer.transform(test_data), columns=test_data.columns)

# 5. 데이터 스케일링 (StandardScaler 적용)
scaler = StandardScaler()
train_features_scaled = pd.DataFrame(scaler.fit_transform(train_features_imputed), 
                                       columns=train_features_imputed.columns)
test_data = pd.DataFrame(scaler.transform(test_data), columns=test_data.columns)

# 6. 불균형 데이터 처리 (SMOTE 적용)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(train_features_scaled, train_target)
# 재조합: SMOTE 처리된 데이터프레임을 train_data로 재정의합니다.
train_data = pd.concat([pd.DataFrame(X_resampled, columns=train_features_scaled.columns), 
                        pd.DataFrame(y_resampled, columns=[target])], axis=1)

In [6]:
train_data.head()

,연간 소득,현재 직장 근속 연수,체납 세금 압류 횟수,개설된 신용계좌 수,신용 거래 연수,최대 신용한도,신용 문제 발생 횟수,마지막 연체 이후 경과 개월 수,개인 파산 횟수,현재 대출 잔액,...,대출 목적_여행 자금,대출 목적_의료비,대출 목적_이사 비용,대출 목적_자동차 구매,대출 목적_주택 개보수,대출 목적_주택 구매,대출 목적_휴가 비용,대출 상환 기간_장기 상환,연체 없음,채무 불이행 여부
0,-0.155206,-0.965030,-0.279027,-0.703173,-0.899120,-0.482924,-0.507403,-0.344278,0.742870,-0.165014,...,-0.132682,-0.126703,-0.020004,-0.097934,-0.31621,-0.051057,-0.024502,-0.658553,-0.063372,0
1,-0.128597,-0.965030,-0.279027,-1.568910,-0.663217,-0.507808,-0.507403,-0.993935,-0.442309,1.426284,...,-0.132682,-0.126703,-0.020004,-0.097934,-0.31621,-0.051057,-0.024502,-0.658553,-0.063372,0
2,-0.563024,0.537992,-0.279027,-0.053870,-0.149779,-0.426087,0.302886,2.154402,3.113228,-1.077898,...,-0.132682,-0.126703,-0.020004,-0.097934,-0.31621,-0.051057,-0.024502,-0.658553,-0.063372,1
3,-0.777310,1.139201,-0.279027,0.595432,-0.704847,-0.476099,0.302886,-0.444225,0.742870,-0.911207,...,-0.132682,-0.126703,-0.020004,-0.097934,-0.31621,-0.051057,-0.024502,-0.658553,-0.063372,1
4,1.503292,-0.063217,-0.279027,-0.270305,0.863219,-0.174536,-0.507403,0.055511,-0.442309,1.678079,...,-0.132682,-0.126703,-0.020004,-0.097934,-0.31621,-0.051057,-0.024502,1.518481,-0.063372,0


In [7]:
# 타깃 변수 지정
label = '채무 불이행 여부'

# AutoGluon TabularPredictor 초기화 및 학습
predictor = TabularPredictor(label=label, eval_metric='roc_auc').fit(
    train_data,
    presets='best_quality',       # 높은 성능을 위한 프리셋 사용
    num_bag_folds=5,              # 교차 검증 폴드 수 증가
    num_stack_levels=2,           # 스택 앙상블 레벨 추가
    time_limit=3600               # 학습에 충분한 시간 할당 (예: 1시간)
)

# 테스트 데이터에 대해 예측 (각 클래스별 확률 예측)
preds_proba = predictor.predict_proba(test_data)
test_data['채무 불이행 확률'] = preds_proba[1]  # 1번 클래스(채무 불이행)의 확률 사용

No path specified. Models will be saved in: "AutogluonModels/ag-20250322_210314"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       30.16 GB / 31.35 GB (96.2%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=5, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fi

(_ray_fit pid=461) [1000]	valid_set's binary_logloss: 0.473331
(_ray_fit pid=605) [1000]	valid_set's binary_logloss: 0.470261 [repeated 5x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


(_dystack pid=248) 	0.8542	 = Validation score   (roc_auc)
(_dystack pid=248) 	19.73s	 = Training   runtime
(_dystack pid=248) 	2.47s	 = Validation runtime
(_dystack pid=248) Fitting model: LightGBM_BAG_L1 ... Training model for up to 374.09s of the 871.85s of remaining time.
(_dystack pid=248) 	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=0.07%)
(_dystack pid=248) 	0.8601	 = Validation score   (roc_auc)
(_dystack pid=248) 	10.58s	 = Training   runtime
(_dystack pid=248) 	0.53s	 = Validation runtime
(_dystack pid=248) Fitting model: RandomForestGini_BAG_L1 ... Training model for up to 360.31s of the 858.07s of remaining time.
(_dystack pid=248) 	0.872	 = Validation score   (roc_auc)
(_dystack pid=248) 	3.62s	 = Training   runtime
(_dystack pid=248) 	0.49s	 = Validation runtime
(_dystack pid=248) Fitting model: RandomForestEntr_BAG_L1 ... Training model for up to 355.91s of the 853.67s of remaining time.
(_d

(_ray_fit pid=2236) [1000]	valid_set's binary_logloss: 0.450862
(_ray_fit pid=2243) [1000]	valid_set's binary_logloss: 0.459506
(_ray_fit pid=2245) [2000]	valid_set's binary_logloss: 0.435687 [repeated 4x across cluster]
(_ray_fit pid=2411) [1000]	valid_set's binary_logloss: 0.436376
(_ray_fit pid=2411) [2000]	valid_set's binary_logloss: 0.433254


(_dystack pid=248) 	0.8644	 = Validation score   (roc_auc)
(_dystack pid=248) 	23.96s	 = Training   runtime
(_dystack pid=248) 	2.77s	 = Validation runtime
(_dystack pid=248) Fitting model: NeuralNetFastAI_r191_BAG_L1 ... Training model for up to 177.28s of the 675.04s of remaining time.
(_dystack pid=248) 	Fitting 5 child models (S1F1 - S1F5) | Fitting with ParallelLocalFoldFittingStrategy (4 workers, per: cpus=1, gpus=0, memory=0.08%)
(_ray_fit pid=2455) No improvement since epoch 13: early stopping
(_ray_fit pid=2454) No improvement since epoch 19: early stopping [repeated 2x across cluster]
(_ray_fit pid=2649) No improvement since epoch 12: early stopping [repeated 2x across cluster]
(_dystack pid=248) 	0.7684	 = Validation score   (roc_auc)
(_dystack pid=248) 	75.41s	 = Training   runtime
(_dystack pid=248) 	0.48s	 = Validation runtime
(_dystack pid=248) Fitting model: CatBoost_r9_BAG_L1 ... Training model for up to 98.78s of the 596.54s of remaining time.
(_dystack pid=248) 	Fitt

...

(_ray_fit pid=3077) 	Ran out of time, early stopping on iteration 3628. Best iteration is: [repeated 4x across cluster]
(_ray_fit pid=3077) 	[3622]	valid_set's binary_logloss: 0.486572 [repeated 4x across cluster]
(_dystack pid=248) 	0.8335	 = Validation score   (roc_auc)
(_dystack pid=248) 	19.17s	 = Training   runtime
(_dystack pid=248) 	3.67s	 = Validation runtime
(_dystack pid=248) Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.00s of the 488.88s of remaining time.
(_dystack pid=248) 	Ensemble Weights: {'ExtraTreesGini_BAG_L1': 0.261, 'ExtraTreesEntr_BAG_L1': 0.261, 'KNeighborsDist_BAG_L1': 0.174, 'CatBoost_r177_BAG_L1': 0.13, 'CatBoost_BAG_L1': 0.087, 'LightGBMLarge_BAG_L1': 0.043, 'CatBoost_r9_BAG_L1': 0.043}
(_dystack pid=248) 	0.8945	 = Validation score   (roc_auc)
(_dystack pid=248) 	0.69s	 = Training   runtime
(_dystack pid=248) 	0.0s	 = Validation runtime
(_dystack pid=248) Fitting 108 L2 models, fit_strategy="sequential" ...
(_dystack pid=248) Fitting m

(_ray_fit pid=5577) [1000]	valid_set's binary_logloss: 0.350941 [repeated 3x across cluster]


(_ray_fit pid=5574) 	Ran out of time, early stopping on iteration 804. Best iteration is:
(_ray_fit pid=5574) 	[804]	valid_set's binary_logloss: 0.336198


(_ray_fit pid=5710) [1000]	valid_set's binary_logloss: 0.331165


(_ray_fit pid=5710) 	Ran out of time, early stopping on iteration 1400. Best iteration is: [repeated 4x across cluster]
(_ray_fit pid=5710) 	[1372]	valid_set's binary_logloss: 0.329364 [repeated 4x across cluster]
(_dystack pid=248) 	0.9207	 = Validation score   (roc_auc)
(_dystack pid=248) 	12.81s	 = Training   runtime
(_dystack pid=248) 	1.23s	 = Validation runtime
(_dystack pid=248) Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.00s of the 154.98s of remaining time.
(_dystack pid=248) 	Ensemble Weights: {'CatBoost_r177_BAG_L2': 0.32, 'XGBoost_BAG_L2': 0.28, 'NeuralNetFastAI_r191_BAG_L2': 0.2, 'NeuralNetFastAI_BAG_L2': 0.12, 'LightGBM_BAG_L2': 0.04, 'LightGBMLarge_BAG_L2': 0.04}
(_dystack pid=248) 	0.9285	 = Validation score   (roc_auc)
(_dystack pid=248) 	0.61s	 = Training   runtime
(_dystack pid=248) 	0.0s	 = Validation runtime
(_dystack pid=248) Fitting 108 L3 models, fit_strategy="sequential" ...
(_dystack pid=248) Fitting model: LightGBMXT_BAG_L3 ... Trainin

In [8]:
# 제출 파일 불러오기 및 예측 결과 반영
submission = pd.read_csv('/kaggle/input/non-fulfillment/sample_submission.csv')
submission['채무 불이행 확률'] = test_data['채무 불이행 확률']
submission.to_csv('submission_autogluon.csv', index=False)

print("AutoGluon을 이용한 예측 및 제출 파일 생성 완료!")

AutoGluon을 이용한 예측 및 제출 파일 생성 완료!
